In [4]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage.exposure import match_histograms

#================= Histogram Calculation ==============================
def histogram(img_2D):
    # Ensure image is uint8 before computing histogram
    if img_2D.dtype != np.uint8:
        img_2D = cv2.normalize(img_2D, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    return cv2.calcHist([img_2D], [0], None, [256], [0, 256]).flatten()

def pdf_f(hist):
    return hist / hist.sum()

def cdf_f(pdf):
    return np.cumsum(pdf)

#================= Mapping-based Histogram Matching ===================
def match_histogram_custom(src_cdf, ref_cdf):
    mapping = np.zeros(256, dtype=np.uint8)
    for src_val in range(256):
        diff = np.abs(ref_cdf - src_cdf[src_val])
        mapping[src_val] = np.argmin(diff)
    return mapping

def img_conv(img_gray, mapping):
    return mapping[img_gray]

#================= Alternative Custom Mapping Method ===================
def custom_match_v2(src_cdf, ref_cdf):
    return np.interp(src_cdf, ref_cdf, np.arange(256)).astype(np.uint8)

#================= Display Function ===================================
def display_results(title, src, ref, matched_builtin, matched_custom, matched_custom_v2):
    # Normalize all to uint8 for saving
    imgs = [src, ref, matched_builtin, matched_custom, matched_custom_v2]
    imgs = [cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8) for img in imgs]
    labels = ["Source", "Reference", "Matched_Skimage", "Matched_Custom", "Matched_Custom_v2"]

    # Save images
    for img, lbl in zip(imgs, labels):
        cv2.imwrite(f"{lbl.replace(' ', '_')}.png", img)

    # Save histograms
    for img, lbl in zip(imgs, labels):
        plt.figure()
        plt.plot(histogram(img), color='black')
        plt.title(f"{lbl} Histogram")
        plt.tight_layout()
        plt.savefig(f"{lbl.replace(' ', '_')}_hist.png")
        plt.close()

    # Save combined grid (optional)
    plt.figure(figsize=(20, 10))
    for i, (img, lbl) in enumerate(zip(imgs, labels)):
        plt.subplot(2, 5, i + 1)
        plt.imshow(img, cmap="gray")
        plt.axis('off')
        plt.title(lbl)
        plt.subplot(2, 5, i + 6)
        plt.plot(histogram(img), color='black')
        plt.title(f"{lbl} Histogram")
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.savefig(f"results_grid_{title.replace(':', '').replace('|', '_').replace(' ', '_')}.png")
    plt.close()

#================= Generate Contrast Variants =========================
def adjust_contrast(img, level="normal"):
    if level == "low":
        return cv2.normalize(img, None, 100, 150, cv2.NORM_MINMAX)  # compress range
    elif level == "high":
        return cv2.equalizeHist(img)  # histogram equalization
    else:
        return img  # normal

#================= Main ===============================================
def main():
    src_path = '/home/zahin/Desktop/DIP_LAB/images/hist_source.jpg'
    ref_path = '/home/zahin/Desktop/DIP_LAB/images/hist_ref.jpeg'
    src_img = cv2.imread(src_path, 0)
    ref_img = cv2.imread(ref_path, 0)

    if src_img is None or ref_img is None:
        print("Error: Could not load images. Check the paths.")
        return

    # Contrast cases
    contrast_levels = ["low", "normal", "high"]

    for src_level in contrast_levels:
        for ref_level in contrast_levels:
            src_mod = adjust_contrast(src_img, src_level)
            ref_mod = adjust_contrast(ref_img, ref_level)

            # Built-in scikit-image histogram matching
            matched_builtin = match_histograms(src_mod, ref_mod)

            # Custom mapping method 1
            src_hist = histogram(src_mod)
            ref_hist = histogram(ref_mod)
            src_cdf = cdf_f(pdf_f(src_hist))
            ref_cdf = cdf_f(pdf_f(ref_hist))
            mapping = match_histogram_custom(src_cdf, ref_cdf)
            matched_custom = img_conv(src_mod, mapping)

            # Custom mapping method 2 (interpolation)
            mapping_v2 = custom_match_v2(src_cdf, ref_cdf)
            matched_custom_v2 = img_conv(src_mod, mapping_v2)

            # Show results
            title = f"Source: {src_level} contrast | Reference: {ref_level} contrast"
            display_results(title, src_mod, ref_mod, matched_builtin, matched_custom, matched_custom_v2)

#================= Run Script =========================================
if __name__ == "__main__":
    main()
